# Check input,output dimensions for FCN

In [2]:
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import SimpleITK as sitk
import scipy.misc as misc
import scipy.ndimage as snd
import imageio
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
%matplotlib inline

## Example Network

In [3]:
class Net1(nn.Module):
    
    def __init__(self):
        
        super(Net1, self).__init__()
        
        self.conv1 = nn.Sequential(nn.Conv2d(1, 32, kernel_size=3, padding=1), nn.BatchNorm2d(32))
        self.conv2 = nn.Sequential(nn.Conv2d(32, 64, kernel_size=3, padding=1), nn.BatchNorm2d(64))
        
        self.conv3 = nn.Sequential(nn.Conv2d(64, 128, kernel_size=3, padding=1), nn.BatchNorm2d(128))
        self.conv4 = nn.Conv2d(128, 2, kernel_size=3, padding=1)

    def forward(self, x):
        
        x = F.relu(self.conv2(F.relu(self.conv1(x))), inplace=True)
        x = F.max_pool2d(x, 2)
        
        x = F.relu(self.conv4(F.relu(self.conv3(x))), inplace=True)
        x = F.max_pool2d(x, 2)

        x = F.log_softmax(x, dim=1)
        
        return x

In [4]:
inputs = torch.randn(1,1,200,200)
inputs = Variable(inputs, volatile = True)
model = Net1()
outputs = model(inputs)
print('Inputs size : ', inputs.size())
print('Outputs size : ',outputs.size())

/Users/pymacbit/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


Inputs size :  torch.Size([1, 1, 200, 200])
Outputs size :  torch.Size([1, 2, 50, 50])


We observe that there is lot of loss in output image resolution when we try to apply a classification network directly on the input image to generate segmentation network